In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import requests
import json

# Import API key

from config import open_weather_api_key
yelp_api_key = "lXiF43rUZ1myoC8YfhKc78q58oC-VfbCIkZ5u-GinwiM-GUBbnVtuXu3vphVyMQoPeCF6iSTkj8YgoSrM-OB1YRCt2OUJSoUV-I_b1XFNQ-7iUIvGk9f6040zBsUXXYx"

In [2]:
print(yelp_api_key)
#print(open_weather_api_key)

lXiF43rUZ1myoC8YfhKc78q58oC-VfbCIkZ5u-GinwiM-GUBbnVtuXu3vphVyMQoPeCF6iSTkj8YgoSrM-OB1YRCt2OUJSoUV-I_b1XFNQ-7iUIvGk9f6040zBsUXXYx


In [3]:
# Save file path to variable
csv_path = "Spicy foods per city.csv"

# Read with Pandas
foods_per_data_df = pd.read_csv(csv_path)
foods_per_data_df = foods_per_data_df.replace(np.nan, 0)
foods_per_data_df.head()

,City,thai,indpak,chinese,colombian,srilankan,caribbean,korean,mexican,laotian,indonesian,ethiopian,malaysian,cajun,senegalese,Test
0,New York,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Los Angeles,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Chicago,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Houston,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Phoenix,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
cuisine_type=[]
cuisine_type =foods_per_data_df.columns[1:].values.tolist()
print(cuisine_type)

['thai', 'indpak', 'chinese', 'colombian ', 'srilankan', 'caribbean', 'korean', 'mexican', 'laotian', 'indonesian', 'ethiopian', 'malaysian', 'cajun', 'senegalese', 'Test']


In [5]:
#Convert Cuisine type to sting to use in api call
cuisine_list=str(cuisine_type)

#remove all unnecesary characters to pass yelp
cuisine_list=cuisine_list.replace("'","")
cuisine_list=cuisine_list.replace("[","")
cuisine_list=cuisine_list.replace("]","")
cuisine_list=cuisine_list.replace(" ","")
print(cuisine_list)

thai,indpak,chinese,colombian,srilankan,caribbean,korean,mexican,laotian,indonesian,ethiopian,malaysian,cajun,senegalese,Test


In [6]:
# params dictionary to update each iteration
url = 'https://api.yelp.com/v3/businesses/search'
headers = {'Authorization': 'bearer %s' % yelp_api_key}


# Create lists to append the API data

city = []
name = []
categories =[]
price = []
rating = []
review_count = []
zip_code = []

In [7]:
master_data = [name,city]
master_data_df = pd.DataFrame({"city": city, 
                               "Restaurant Name": name,
                               "Cuisine" : categories,
                               "Rating" : rating,
                               "Num of Reviews" : review_count,
                               "Zip Code" : zip_code
                              })
import master_data_df

,city,Restaurant Name,Cuisine,Rating,Num of Reviews,Zip Code


In [53]:
import numpy

record = 1

# Log file print statement
print("Beginning Data Retrieval")
print("-------------------------------")

#rows are cities
NY = 0
LA = 1
CHIC = 2
HSTN = 3
PHNX = 4
MAX_CITY = 5 # to limit the number of cities

# coloumns in city_rest_details
THAI = 0
CHINA = 1
MALAY = 2
MEX = 3
COL = 4
SL = 5
CAR = 6
IND = 7 # ind and pak
MAX_REST = 8

city_rest_details = [[0] * MAX_REST for _ in range(MAX_CITY)]
# city_rest_details[MAX_CITY][MAX_REST]

#Create for loop to loop through all cities
# Use the lat/lng we recovered to identify airports
for index, row in foods_per_data_df.iterrows():
    for cuisine in foods_per_data_df.columns[1:]:
    
    # change location each iteration while leaving original params in place
    #changing to Austin to test changeback to row[0]
            params = {"location" : row[0],
                    'categories': cuisine_list,
                     'limit' : 50
                     }

    #base URL
    url = 'https://api.yelp.com/v3/businesses/search'

    # make request and print url
    # try:
    response = requests.get(url, params=params, headers=headers)
    # print(response.text)

# convert to json
    business_data = json.loads(response.text)
    
    
    for item in business_data["businesses"]:
        # print(item["location"]["city"])
        if item["location"]["city"] == "New York":
            city = NY
        elif item["location"]["city"] == "Los Angeles":
            city = LA
        elif item["location"]["city"] == "Chicago":
            city = CHIC
        elif item["location"]["city"] == "Houston":
            city = HSTN
        elif item["location"]["city"] == "Phoenix":
            city = PHNX
        else:
            city = MAX
        
        if city < MAX:
            for cat in item["categories"]:
                if cat["title"] == "Thai":
                    city_rest_details[city] [THAI] = 1 
                elif cat["title"] == "Chinese":     
                    city_rest_details[city][CHINA] += 1 
                elif cat["title"] == "Malaysian":
                    city_rest_details[city][MALAY] += 1 
                elif cat["title"] == "Mexican":
                    city_rest_details[city][MEX] += 1 
                elif cat["title"] == "columbian":
                    city_rest_details[city][COL] += 1 
                elif cat["title"] == "Srilankan":
                    city_rest_details[city][SL] += 1 
                elif cat["title"] == "Caribbean":
                    city_rest_details[city][CAR] += 1
                elif cat["title"] == "Indian" or cat["title"] == "Pakistani":
                    city_rest_details[city][IND] += 1 
                # else:
                #    print(cat["title"])
                
#for i,j in city_rest_details:
#    print(city_rest_details[i][j])
print(numpy.matrix(city_rest_details))

#print("chinese restaurants: " + str(rest_chinese))
#print("Thai restaurants: " + str(rest_thai))
#print("Malaysian restaurants: " + str(rest_malaysian))
#print("Mexican restaurants: " + str(rest_mexican)) 
#print("columbian restaurants: " + str(rest_columbian))
#print("Srilankan restaurants: " + str(rest_srilankan))
#print("caribbean restaurants: " + str(rest_caribbean))
#print("Indpak restaurants: " + str(rest_indpak))      
   
            

Beginning Data Retrieval
-------------------------------
[[ 1 11  2  5  0  0  0  3]
 [ 1  1  0 13  0  0  0  2]
 [ 1  5  1 19  0  0  0  4]
 [ 1  1  0 21  0  0  1  4]
 [ 1  2  0 33  0  0  2  2]]


In [ ]:
print(business_data['total'])

In [ ]:
print(params)

In [ ]:
#---- try expect area
# Start the call counter 
city = []
name = []
categories =[]
price = []
rating = []
review_count = []
zip_code = []

record = 1

# Log file print statement
print("Beginning Data Retrieval")
print("-------------------------------")

#Create for loop to loop through all cities
# Use the lat/lng we recovered to identify airports
for index, row in foods_per_data_df.iterrows():
    for cuisine in foods_per_data_df.columns[1:]:

    # change location each iteration while leaving original params in place
    #changing to Austin to test changeback to row[0]
            params = {"location" : row[0],
                    'categories': cuisine_list,
                     'limit' : 50,
                    'offset' : 900
                     }

    #base URL
    base_url = 'https://api.yelp.com/v3/businesses/search'
    try:
        # make request and print url
        response = requests.get(url, params=params, headers=headers)


        # convert to json
        business_data = response.json()
        for x in range(len(business_data['businesses'])):       
        # Try statement to append calls where value is found 


            city.append(business_data["businesses"][x]["location"]["city"])
            name.append(business_data['businesses'][x]["name"])
            categories.append(business_data['businesses'][x]["categories"][0]["alias"])
            price.append(business_data['businesses'][0]["price"])
            rating.append(business_data['businesses'][x]["rating"])
            review_count.append(business_data['businesses'][x]["review_count"])
            zip_code.append(business_data['businesses'][x]["location"]["zip_code"])
            #print(f"Processing Record {record} | {location}")
            #print(f"{url}&q={city}")


            # Increase counter by one 
            record = record + 1

            # Setting timer to not over exceed rate limit of API
            #https://openweathermap.org/guide
            #Free tariff The number of calls you are allowed to make depends on the subscription plan you choose. 
            #Free users can make 60 requests per minute. Using the service under Free tier, 
            #you can work with the following weather APIs:
            #time.sleep(1.5)

             #If no record found "skip" to next call
    except:
            print("----City not found, Moving on----")
            continue

business_data

In [54]:
record = 1

# Log file print statement
print("Beginning Data Retrieval")
print("-------------------------------")



rest_thai = 0
rest_chinese = 0
rest_malaysian = 0
rest_mexican = 0
rest_columbian = 0
rest_srilankan = 0
rest_caribbean = 0
rest_indpak = 0


#Create for loop to loop through all cities
# Use the lat/lng we recovered to identify airports
for index, row in foods_per_data_df.iterrows():
    for cuisine in foods_per_data_df.columns[1:]:
    
    # change location each iteration while leaving original params in place
    #changing to Austin to test changeback to row[0]
            params = {"location" : row[0],
                    'categories': cuisine_list,
                     'limit' : 50
                     }

    #base URL
    url = 'https://api.yelp.com/v3/businesses/search'

    # make request and print url
    # try:
    response = requests.get(url, params=params, headers=headers)
    
    # convert to json
    business_data = json.loads(response.text)
    
    
    for item in business_data["businesses"]:
        # print(item["location"]["city"])
        if item["location"]["city"] == "New York":
             for cat in item["categories"]:
                if cat["title"] == "Thai":
                    rest_thai += 1 
                elif cat["title"] == "Chinese":     
                    rest_chinese += 1                     
                elif cat["title"] == "Malaysian":
                    rest_malaysian += 1 
                elif cat["title"] == "Mexican":
                    rest_mexican += 1
                elif cat["title"] == "columbian":
                    rest_columbian += 1 
                elif cat["title"] == "Srilankan":
                    rest_srilankan += 1
                elif cat["title"] == "Caribbean":
                    rest_caribbean += 1
                elif cat["title"] == "Indian" or cat["title"] == "Pakistani":
                    rest_indpak += 1
                # else:
                #    print(cat["title"])
                
#for i,j in city_rest_details:
#    print(city_rest_details[i][j])
#print(numpy.matrix(city_rest_details))

print("chinese restaurants: " + str(rest_chinese))
print("Thai restaurants: " + str(rest_thai))
print("Malaysian restaurants: " + str(rest_malaysian))
print("Mexican restaurants: " + str(rest_mexican)) 
print("columbian restaurants: " + str(rest_columbian))
print("Srilankan restaurants: " + str(rest_srilankan))
print("caribbean restaurants: " + str(rest_caribbean))
print("Indpak restaurants: " + str(rest_indpak))      
   
            

Beginning Data Retrieval
-------------------------------
chinese restaurants: 11
Thai restaurants: 8
Malaysian restaurants: 2
Mexican restaurants: 5
columbian restaurants: 0
Srilankan restaurants: 0
caribbean restaurants: 0
Indpak restaurants: 3
